In [1]:
import os
import shutil
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import (
    VGG16, VGG19, ResNet50, ResNet50V2, ResNet101, ResNet101V2, ResNet152, ResNet152V2,
    InceptionV3, InceptionResNetV2, Xception,
    MobileNet, MobileNetV2,
    DenseNet121, DenseNet169, DenseNet201,
    NASNetMobile
)
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg19_preprocess
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnetv2_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as inceptionresnet_preprocess
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenetv2_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
from tensorflow.keras.applications.nasnet import preprocess_input as nasnet_preprocess
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Set seeds for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Training source directories
train_sources = [
    '/kaggle/input/chestxray-augment/train',
    '/kaggle/input/chest-xray-pneumonia/chest_xray/train',
    '/kaggle/input/dcgan-pneumonia/chest_xray/train'
]

# Test directory (unchanged)
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'

# Combined training directory
combined_train_dir = '/kaggle/working/combined_train'

# Image and training parameters
BATCH_SIZE = 32
EPOCHS = 64
IMG_SIZE = (224, 224)

# Combine all training sources into one directory
if os.path.exists(combined_train_dir):
    shutil.rmtree(combined_train_dir)

os.makedirs(os.path.join(combined_train_dir, 'NORMAL'))
os.makedirs(os.path.join(combined_train_dir, 'PNEUMONIA'))

def copy_images(src_dir, dest_dir):
    for label in ['NORMAL', 'PNEUMONIA']:
        src_label_dir = os.path.join(src_dir, label)
        dest_label_dir = os.path.join(dest_dir, label)
        if os.path.exists(src_label_dir):
            for fname in os.listdir(src_label_dir):
                src_path = os.path.join(src_label_dir, fname)
                dest_path = os.path.join(dest_label_dir, fname)
                if os.path.isfile(src_path):
                    shutil.copy(src_path, dest_path)

# Copy from all sources
for src in train_sources:
    copy_images(src, combined_train_dir)

# Data Generators
def get_generators(preprocess_func):
    train_gen = ImageDataGenerator(
        preprocessing_function=preprocess_func,
        horizontal_flip=True,
        zoom_range=0.2,
        rotation_range=15
    ).flow_from_directory(
        combined_train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        seed=SEED
    )

    test_gen = ImageDataGenerator(
        preprocessing_function=preprocess_func
    ).flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )
    return train_gen, test_gen

# Build classifier head
def build_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

# Models Dictionary
models = {
    'Xception': (Xception(weights='imagenet', include_top=False, input_shape=(224,224,3)), xception_preprocess),
    'VGG16': (VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3)), vgg_preprocess),
    'VGG19': (VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3)), vgg19_preprocess),
    'ResNet50': (ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnet_preprocess),
    'ResNet50V2': (ResNet50V2(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnetv2_preprocess),
    'ResNet101': (ResNet101(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnet_preprocess),
    'ResNet101V2': (ResNet101V2(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnetv2_preprocess),
    'ResNet152': (ResNet152(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnet_preprocess),
    'ResNet152V2': (ResNet152V2(weights='imagenet', include_top=False, input_shape=(224,224,3)), resnetv2_preprocess),
    'InceptionV3': (InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3)), inception_preprocess),
    'InceptionResNetV2': (InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3)), inceptionresnet_preprocess),
    'MobileNet': (MobileNet(weights='imagenet', include_top=False, input_shape=(224,224,3)), mobilenet_preprocess),
    'MobileNetV2': (MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3)), mobilenetv2_preprocess),
    'DenseNet121': (DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3)), densenet_preprocess),
    'DenseNet169': (DenseNet169(weights='imagenet', include_top=False, input_shape=(224,224,3)), densenet_preprocess),
    'DenseNet201': (DenseNet201(weights='imagenet', include_top=False, input_shape=(224,224,3)), densenet_preprocess),
    'NASNetMobile': (NASNetMobile(weights='imagenet', include_top=False, input_shape=(224,224,3)), nasnet_preprocess),
}

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2)

# Result storage
report_rows = []

# Training and Evaluation Loop
for model_name, (base_model, preprocess_func) in models.items():
    print(f"\n🔧 Training model: {model_name}")
    train_gen, test_gen = get_generators(preprocess_func)
    model = build_model(base_model)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(
        train_gen,
        epochs=EPOCHS,
        validation_data=test_gen,
        callbacks=[early_stop, reduce_lr],
        verbose=2
    )

    predictions = model.predict(test_gen, verbose=0)
    predicted_labels = (predictions > 0.5).astype(int).ravel()
    true_labels = test_gen.classes

    accuracy = accuracy_score(true_labels, predicted_labels)
    report = classification_report(true_labels, predicted_labels, digits=6, output_dict=True)

    row = {
        'Model': model_name,
        'Accuracy': accuracy,
        'Macro Precision': report['macro avg']['precision'],
        'Macro Recall': report['macro avg']['recall'],
        'Macro F1': report['macro avg']['f1-score'],
        'Weighted Precision': report['weighted avg']['precision'],
        'Weighted Recall': report['weighted avg']['recall'],
        'Weighted F1': report['weighted avg']['f1-score'],
    }

    report_rows.append(row)

# Save to CSV
df_report = pd.DataFrame(report_rows)
df_report.to_csv('/kaggle/working/classification_summary.csv', index=False)
print("✅ Saved summary to /kaggle/working/classification_summary.csv")
print("📊 Classification Summary:")
print(df_report)


2025-07-21 08:51:51.582684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753087911.799354      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753087911.863297      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1753088053.973705      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
171317808/171317808 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
234698864/234698864 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
234545216/234545216 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
51877672/51877672 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
19993432/19993432 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step

🔧 Training model: Xception
Found 21224 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64


I0000 00:00:1753088152.806345      57 service.cc:148] XLA service 0x7e1bd8002be0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753088152.807683      57 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753088153.872500      57 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753088158.724171      57 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

Epoch 1/64
664/664 - 344s - 517ms/step - accuracy: 0.8945 - loss: 0.2558 - val_accuracy: 0.8766 - val_loss: 0.2859 - learning_rate: 0.0010
Epoch 2/64
664/664 - 260s - 391ms/step - accuracy: 0.9124 - loss: 0.2137 - val_accuracy: 0.8734 - val_loss: 0.3752 - learning_rate: 0.0010
Epoch 3/64
664/664 - 264s - 397ms/step - accuracy: 0.9148 - loss: 0.2072 - val_accuracy: 0.8894 - val_loss: 0.3551 - learning_rate: 0.0010
Epoch 4/64
664/664 - 268s - 404ms/step - accuracy: 0.9224 - loss: 0.1890 - val_accuracy: 0.8990 - val_loss: 0.3050 - learning_rate: 2.0000e-04

🔧 Training model: DenseNet201
Found 21224 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64
664/664 - 363s - 547ms/step - accuracy: 0.8932 - loss: 0.2556 - val_accuracy: 0.8397 - val_loss: 0.4672 - learning_rate: 0.0010
Epoch 2/64
664/664 - 266s - 401ms/step - accuracy: 0.9123 - loss: 0.2146 - val_accuracy: 0.8205 - val_loss: 0.5390 - learning_rate: 0.0010
Epoch 3/64
664/664 - 262s - 395ms/step - accuracy: 0.9172 - loss: 0.2062 - val_accuracy: 0.8926 - val_loss: 0.2901 - learning_rate: 0.0010
Epoch 4/64
664/664 - 261s - 393ms/step - accuracy: 0.9197 - loss: 0.1975 - val_accuracy: 0.8846 - val_loss: 0.3150 - learning_rate: 0.0010
Epoch 5/64
664/664 - 262s - 395ms/step - accuracy: 0.9199 - loss: 0.1961 - val_accuracy: 0.9135 - val_loss: 0.2182 - learning_rate: 0.0010
Epoch 6/64
664/664 - 263s - 396ms/step - accuracy: 0.9210 - loss: 0.1933 - val_accuracy: 0.9054 - val_loss: 0.2617 - learning_rate: 0.0010
Epoch 7/64
664/664 - 263s - 396ms/step - accuracy: 0.9216 - loss: 0.1901 - val_accuracy: 0.8429 - val_loss: 0.5046 - learning_rate: 0.0010
Epoch 8/64
664/664 - 264s -

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64
664/664 - 327s - 493ms/step - accuracy: 0.8768 - loss: 0.2912 - val_accuracy: 0.7788 - val_loss: 0.5526 - learning_rate: 0.0010
Epoch 2/64
664/664 - 247s - 372ms/step - accuracy: 0.8971 - loss: 0.2525 - val_accuracy: 0.8045 - val_loss: 0.4933 - learning_rate: 0.0010
Epoch 3/64
664/664 - 250s - 376ms/step - accuracy: 0.8985 - loss: 0.2443 - val_accuracy: 0.8301 - val_loss: 0.4248 - learning_rate: 0.0010
Epoch 4/64
664/664 - 255s - 384ms/step - accuracy: 0.9011 - loss: 0.2367 - val_accuracy: 0.8077 - val_loss: 0.5354 - learning_rate: 0.0010
Epoch 5/64
664/664 - 252s - 380ms/step - accuracy: 0.9050 - loss: 0.2309 - val_accuracy: 0.8958 - val_loss: 0.2830 - learning_rate: 0.0010
Epoch 6/64
664/664 - 254s - 382ms/step - accuracy: 0.9060 - loss: 0.2271 - val_accuracy: 0.9006 - val_loss: 0.2881 - learning_rate: 0.0010
Epoch 7/64
664/664 - 250s - 377ms/step - accuracy: 0.9074 - loss: 0.2235 - val_accuracy: 0.8942 - val_loss: 0.2928 - learning_rate: 0.0010
Epoch 8/64
664/664 - 247s -